<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и проверка модели</a></span></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span></li></ul></div>

# Выбор локации для скважины

Вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Нужно построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Прочитаем файлы /datasets/geo_data_0.csv,  /datasets/geo_data_1.csv,  /datasets/geo_data_2.csv 
и сохраним их в переменных `geo_0`, `geo_1`, `geo_2` соответственно.

In [2]:
# читаем файл и сохраняем его в bank
geo_1 = pd.read_csv('/datasets/geo_data_0.csv') 

In [3]:
# читаем файл и сохраняем его в bank
geo_2 = pd.read_csv('/datasets/geo_data_1.csv') 

In [4]:
# читаем файл и сохраняем его в bank
geo_3 = pd.read_csv('/datasets/geo_data_2.csv') 

Напишем функцию, которая выводит общую информацию о таблице, а так же посчитает пропуски и проверит на наличие дубликатов.

In [5]:
#пишем функцию summary
def summary(df):
    print(df.info())
    display(df.head())
    count=0
    for col in df.columns:
        if df.isna().sum()[col]==0:
            count+=1
        else:
            print(f'Пропущенных значений в столбце {col}: {df.isna().sum()[col]}')
    if count==len(df.columns):
        print('Пропущенных значений в таблице нет')
    print(f'Повторов в таблице {df.duplicated().sum()}') 

In [6]:
summary(geo_1) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


Пропущенных значений в таблице нет
Повторов в таблице 0


In [7]:
summary(geo_2) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


Пропущенных значений в таблице нет
Повторов в таблице 0


In [8]:
summary(geo_3) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Пропущенных значений в таблице нет
Повторов в таблице 0


Итак есть 3 потенциальных региона для добычи нефти. В каждом регионе есть данные о 100 тысячах предполагаемых скважин для добычи нефти. Данные без пропусков. Без повторов. 

Из условия известно, что:

* id — уникальный идентификатор скважины;
* f0, f1, f2 — три признака точек;
* product — объём запасов в скважине (тыс. баррелей).

Столбец `id` никак не повлияет на прогноз запасов нефти в каждом регионе, тем более что он записан в формате `object`, а модель с ним работать не будет. Не вижу смысла его форматировать. Удалим данный столбец в каждом регионе.

In [9]:
#удалим столбец id
geo_1.drop(['id'], axis=1, inplace=True)

In [10]:
#удалим столбец id
geo_2.drop(['id'], axis=1, inplace=True)

In [11]:
#удалим столбец id
geo_3.drop(['id'], axis=1, inplace=True)

Оценим зависимость целевого признака от признаков. Найдём коэффициенты корреляции.

In [15]:
#исследуем зависимость в регионе 1
geo_1.corr().style.background_gradient(subset=['f0'], 
                                       cmap='Reds_r').background_gradient(subset=['f1'],
                                       cmap='Blues_r').background_gradient(subset=['f2'], 
                                       cmap='Oranges_r').background_gradient(subset=['product'],cmap='Greens')

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


Есть выраженная зависимость от признака f2. От признаков f0 и f1, целевой признак почти не зависит.

In [16]:
# исследуем зависимость в регионе 2
geo_2.corr().style.background_gradient(subset=['f0'], 
                                       cmap='Reds_r').background_gradient(subset=['f1'],
                                       cmap='Blues_r').background_gradient(subset=['f2'], 
                                       cmap='Oranges_r').background_gradient(subset=['product'],cmap='Greens')

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


Сильная линейная зависимость между признаком f2 и целевым признаком. Псокольку сама суть признаков от нас скрыта, то удалять его нельзя. От признаков f0 и f1 целевой признак независит.

In [17]:
# исследуем зависимость в регионе 3
geo_3.corr().style.background_gradient(subset=['f0'], 
                                       cmap='Reds_r').background_gradient(subset=['f1'],
                                       cmap='Blues_r').background_gradient(subset=['f2'], 
                                       cmap='Oranges_r').background_gradient(subset=['product'],cmap='Greens')

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


Есть выраженная зависимость от признака f2. От признаков f0 и f1, целевой признак не зависит.

На основе полученных коэффициентов, можно сделать вывод, что признаки f0 и f1 не оказывают влияния на целевой признак. В то же самое время признак f2 в двух регионах оказывает умеренную элептическую зависимость. А в регионе 2 сильную линейную зависимость.

**Вывод**
* были открыты и узучены данные о запасах нефти в 3-х регионах
* данные без пропусков и повторов
* была изучена зависимость целевого признака от признаков для каждого региона:
 * для 1-го и 3-го региона целевоц признак имеет умеренную элептическую зависимость от признака f2(коэ-т кор-ии около 0,5)
 * для 2-го региона целевоц признак имеет сильную линейную зависимость  от признака f2 (коэ-т кор-ии 0,999)

## Обучение и проверка модели

Напишем функцию `get_predictions(geo)`. Для каждого региона в ней зададим целевой признак и признаки. Произведём разбивку данных на обучающую и валидационную в соотношении 75:25. Обучим модель линейной регрессии. Функция будет возвращать правильные ответы и полученные предсказания на валидационной выборке.

In [18]:
# пишем функцию get_predictions(geo)
# задаём признаки и целевой признак
# производим разбивку данных
# задаём модель, обучаем её и получаем предсказания
def get_predictions(geo):
    features = geo.drop(['product'], axis=1) 
    target = geo['product']
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target , test_size=0.25, random_state=12345)
    model = LinearRegression() 
    model.fit(features_train, target_train)
    return pd.Series(model.predict(features_valid), index=target_valid.index), target_valid

Сохраним предсказания и правильные ответы для каждого региона.

In [19]:
# сохраним предсказания и правильные ответы для 1-го региона
predicted_1, target_1 = get_predictions(geo_1)

In [20]:
# сохраним предсказания и правильные ответы для 2-го региона
predicted_2, target_2 = get_predictions(geo_2)

In [21]:
# сохраним предсказания и правильные ответы для 3-го региона
predicted_3, target_3 = get_predictions(geo_3)

Найдём средний предсказанный запас сырья для каждого региона.

In [22]:
# найдём средний предсказанный запас сырья для 1-го региона
predicted_1.mean()

92.59256778438035

In [23]:
# найдём средний предсказанный запас сырья для 2-го региона
predicted_2.mean()

68.728546895446

In [24]:
# найдём средний предсказанный запас сырья для 3-го региона
predicted_3.mean()

94.96504596800489

Вычислим RMSE для каждого региона

In [25]:
# вычисляем rmse для 1-го региона
rmse_1 = mean_squared_error(target_1, predicted_1)**0.5

In [26]:
# вычисляем rmse для 2-го региона
rmse_2 = mean_squared_error(target_2, predicted_2)**0.5

In [27]:
# вычисляем rmse для 3-го региона
rmse_3 = mean_squared_error(target_3, predicted_3)**0.5

Объеденим полученные результаты в таблицу.

In [28]:
res=pd.DataFrame(columns=['predictions', 'rmse'])

In [29]:
res=res.append({"predictions":predicted_1.mean(), 'rmse':rmse_1}, ignore_index=True)
res=res.append({"predictions":predicted_2.mean(), 'rmse':rmse_2}, ignore_index=True)
res=res.append({"predictions":predicted_3.mean(), 'rmse':rmse_3}, ignore_index=True)
display(res.head(3))

,predictions,rmse
0,92.592568,37.579422
1,68.728547,0.893099
2,94.965046,40.029709


Из таблицы видно, что средние предсказанные запасы в 1-ом и 3-ем регионах примерно равны и составляют 92 - 94 тыс. баррелей нефти в скважине. А во 2-ом регионе средний предсказанный запас - 69 тыс. баррелей. Но при этом корень из среднееквадратичной ошибки самый наименьший у второго региона и равен 0.893099. А чем ближе значение rmse к 0, тем точнее модель предсказывает значения. Столько малое значение rmse, по всей видимости, связано с тем, что во 2-ом регионе есть прямая зависимость между признаком f2 и целевым признаком. Так же можно сделать вывод, что модель по 1-му и 3-му региону сильно переоценивает запасы.

**Вывод**
* была написана функция, которая для каждого региона:
 * задаёт признаки и целевой признак
 * разбивает данные на обучающую и валидационную выборки
 * обучает модель и делает предсказания
 * возвращает правильные ответы и предсказания
* были вычислены средние предсказанные запасы для каждого региона
* было подсчитаны rmse для каждого региона.

## Подготовка к расчёту прибыли

Сохраним все ключевые значения для расчётов в отдельных переменных.Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе. Из условия задачи известно:
 * при разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки;
 * бюджет на разработку скважин в регионе — 10 млрд рублей;
 * при нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

In [30]:
# сохраним количество исследуемых точек
WELL = 500

In [31]:
# сохраним количество лучших скважин
TOP_WELL = 200

In [32]:
# сохраним значение бюджета на регион
BUDGET = 10000000000

In [33]:
# сохраним доход с 1000 бареллей
REVENUE = 450000

Расчитаем достаточный объём сырья, для безубыточной разработки скважин.

In [34]:
# расчитываем необходимый запас сырья для разработки
ENOUGHT_CRUDE = BUDGET/(TOP_WELL*REVENUE)
display('Достаточный объём сырья, тыс. баррелей', round(ENOUGHT_CRUDE))

'Достаточный объём сырья, тыс. баррелей'

111

Для безубыточной разработки скважин в каждом регионе нужно пробурить 200 скважин с запасом в каждой около 111 тысяч баррелей. Только 2-ой и 3-ий регион близки к этому значению, по среднему предсказанному запасу. Но у этих предсказаний средняя квадратичная ошибка слишком велика. Продолжим исследование. Посчитаем прибыли по выбранным скважинам и предсказаниям модели. Посчитаем риски для каждого региона.

**Вывод**
* были сохранены все ключевые значения для расчётов прибыли в отдельных переменных
* был расчитан необходимы запас сырья для безубыточной разработки скважины в регионе, он равен: 111 тыс.бареллей

## Расчёт прибыли и рисков 

Напишем функцию для расчета прибыли.

In [35]:
def profit(target, predictions):
    profit = 0
    pred_sorted = predictions.sort_values(ascending=False).head(TOP_WELL)
    selected = target[pred_sorted.index][:TOP_WELL]
    
    for WELL in selected:
        profit += (WELL - ENOUGHT_CRUDE)*REVENUE
    return profit    

In [36]:
def bootstrap(target, predictions):
    state = np.random.RandomState(12345)
    values = []
    for i in range(1000):
        target_subsample = target.sample(n=WELL, replace=True, random_state=state)
        probs_subsample = predictions[target_subsample.index]
        values.append(profit(target_subsample, probs_subsample))
        
    values = pd.Series(values)
    lower = values.quantile(0.025)/1000000
    upper = values.quantile(0.975)/1000000
    mean = values.mean()/1000000
    risk = (values < 0).mean()
    
    print(f'Средняя выручка:, {mean:.2f} млн.руб')
    print(f'Доверительный интервал от, {lower:.2f} млн.руб до {upper:.2f}')
    print(f'Риск убытка:, {risk:.1%}')

Найдём значения для каждого региона.

In [37]:
# вычислим значения для 1-го региона
print(bootstrap(target_1, predicted_1))

Средняя выручка:, 425.94 млн.руб
Доверительный интервал от, -102.09 млн.руб до 947.98
Риск убытка:, 6.0%
None


In [38]:
# вычислим значения для 2-го региона
print(bootstrap(target_2, predicted_2))

Средняя выручка:, 515.22 млн.руб
Доверительный интервал от, 68.87 млн.руб до 931.55
Риск убытка:, 1.0%
None


In [39]:
# вычислим значения для 3-го региона
print(bootstrap(target_3, predicted_3))

Средняя выручка:, 435.01 млн.руб
Доверительный интервал от, -128.88 млн.руб до 969.71
Риск убытка:, 6.4%
None


**Вывод**
* лучшая средняя выручка во 2-ом регионе - 515.22 млн.руб.
* риск убытка ниже всех во втором регионе - 1.0%
* на основе полученных результатов, 2-ой регион рекомендован к разработке скважин